# Routing 模式演示

本notebook演示了Routing（路由）模式，这是一种智能任务分配技术。
一个LLM作为路由器（Router）来分析任务类型，然后将任务分配给最适合的专门模型处理。

**工作流程：**
1. Router LLM 分析用户请求，判断任务类型
2. 根据任务类型路由到相应的专门模型：
   - 技术/代码类任务 → deepseek-chat（擅长编程）
   - 创意/写作类任务 → claude-sonnet-4-5（擅长创意写作）
3. 专门模型处理任务并返回结果

**优势：**
- 关注点分离
- 充分发挥各模型的专长
- 提高响应质量和效率

## 1、加载密钥环境OpenRouter

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

load_dotenv(override=True)
api_key = os.getenv("OPENROUTER_API_KEY")
if api_key:
    print("API Key loaded successfully.")
else:
    print("Failed to load API Key.")

API Key loaded successfully.


## 2、初始化OpenRouter客户端

In [2]:
openrouter = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=api_key)
print("OpenRouter客户端初始化成功")

OpenRouter客户端初始化成功


## 3、定义专门模型及其擅长领域

配置两个专门的模型，每个模型都有其擅长的任务类型

In [3]:
# 定义专门模型配置
SPECIALIST_MODELS = {
    "technical": {
        "model": "deepseek/deepseek-chat",
        "name": "DeepSeek技术专家",
        "description": "擅长处理编程、技术分析、算法设计、代码调试等技术类任务",
        "keywords": ["代码", "编程", "算法", "技术", "开发", "函数", "bug", "调试", "数据结构"]
    },
    "creative": {
        "model": "anthropic/claude-sonnet-4-5",
        "name": "Claude创意专家",
        "description": "擅长处理创意写作、内容创作、故事构思、文案策划等创意类任务",
        "keywords": ["写作", "故事", "创意", "文案", "营销", "广告", "诗歌", "小说", "剧本"]
    }
}

print("=== 专门模型配置 ===")
for task_type, config in SPECIALIST_MODELS.items():
    print(f"\n【{task_type.upper()}】{config['name']}")
    print(f"模型: {config['model']}")
    print(f"专长: {config['description']}")

=== 专门模型配置 ===

【TECHNICAL】DeepSeek技术专家
模型: deepseek/deepseek-chat
专长: 擅长处理编程、技术分析、算法设计、代码调试等技术类任务

【CREATIVE】Claude创意专家
模型: anthropic/claude-sonnet-4-5
专长: 擅长处理创意写作、内容创作、故事构思、文案策划等创意类任务


## 4、定义Router函数

Router使用 **gpt-5-mini** 作为路由器，分析任务并决定使用哪个专门模型

In [5]:
def route_task(user_request):
    """
    Router函数：分析用户请求并决定使用哪个专门模型
    
    参数:
        user_request: 用户的原始请求
    
    返回:
        task_type: 'technical' 或 'creative'
        reasoning: 路由决策的理由
    """
    print("\n=== Router 正在分析任务 ===")
    print(f"用户请求: {user_request}\n")
    
    # 构建Router提示词
    router_prompt = f"""你是一个任务路由器。分析以下用户请求，判断它属于哪种类型：

用户请求：
{user_request}

可选的任务类型：
1. technical - 技术类任务（编程、代码、算法、技术分析等）
2. creative - 创意类任务（写作、故事、文案、营销内容等）

请以JSON格式回答，只返回JSON，不要包含其他内容：
{{
    "task_type": "technical 或 creative",
    "confidence": "0-100的置信度",
    "reasoning": "分析理由"
}}"""
    
    # 调用Router模型
    router_model = "gpt-5-mini"
    response = openrouter.chat.completions.create(
        model=router_model,
        messages=[{"role": "user", "content": router_prompt}]
    )
    
    # 解析Router的决策
    router_output = response.choices[0].message.content
    print(f"Router模型 ({router_model}) 的原始输出:")
    print(router_output)
    
    # 提取JSON
    try:
        # 尝试直接解析
        decision = json.loads(router_output)
    except:
        # 如果包含markdown代码块，提取JSON部分
        import re
        json_match = re.search(r'\{[^}]+\}', router_output, re.DOTALL)
        if json_match:
            decision = json.loads(json_match.group())
        else:
            # 默认路由
            decision = {"task_type": "creative", "confidence": 50, "reasoning": "无法解析，使用默认路由"}
    
    task_type = decision.get("task_type", "creative")
    confidence = decision.get("confidence", 0)
    reasoning = decision.get("reasoning", "未提供理由")
    
    print(f"\n✓ 路由决策: {task_type.upper()}")
    print(f"✓ 置信度: {confidence}%")
    print(f"✓ 理由: {reasoning}")
    
    return task_type, reasoning

## 5、定义任务执行函数

根据路由决策，使用相应的专门模型执行任务

In [6]:
def execute_task(user_request, task_type):
    """
    使用专门模型执行任务
    
    参数:
        user_request: 用户的原始请求
        task_type: Router决定的任务类型
    
    返回:
        result: 专门模型的输出结果
    """
    print(f"\n=== 使用专门模型处理任务 ===")
    
    # 获取专门模型配置
    specialist = SPECIALIST_MODELS[task_type]
    model = specialist["model"]
    name = specialist["name"]
    
    print(f"专门模型: {name}")
    print(f"模型ID: {model}")
    print(f"任务类型: {task_type.upper()}\n")
    
    # 调用专门模型
    response = openrouter.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": user_request}]
    )
    
    result = response.choices[0].message.content
    return result, model, name

## 6、测试案例1：技术类任务

测试一个编程相关的技术任务，观察Router如何将其路由到DeepSeek模型

In [7]:
print("\n" + "="*70)
print("测试案例1：技术类任务")
print("="*70)

# 技术类请求
technical_request = """请用Python编写一个函数，实现二分查找算法。
要求包含详细注释，并说明时间复杂度。"""

# 步骤1: Router分析任务
task_type_1, reasoning_1 = route_task(technical_request)

# 步骤2: 执行任务
result_1, model_1, name_1 = execute_task(technical_request, task_type_1)

# 显示结果
print(f"\n{'='*70}")
print(f"执行结果 - {name_1}")
print("="*70)
display(Markdown(result_1))


测试案例1：技术类任务

=== Router 正在分析任务 ===
用户请求: 请用Python编写一个函数，实现二分查找算法。
要求包含详细注释，并说明时间复杂度。

Router模型 (gpt-5-mini) 的原始输出:
{
  "task_type": "technical",
  "confidence": "100",
  "reasoning": "用户请求明确要求用Python编写函数实现二分查找算法，包含详细注释并说明时间复杂度，属于编程/算法实现范畴，属于技术类任务，不涉及创意写作或营销类内容。"
}

✓ 路由决策: TECHNICAL
✓ 置信度: 100%
✓ 理由: 用户请求明确要求用Python编写函数实现二分查找算法，包含详细注释并说明时间复杂度，属于编程/算法实现范畴，属于技术类任务，不涉及创意写作或营销类内容。

=== 使用专门模型处理任务 ===
专门模型: DeepSeek技术专家
模型ID: deepseek/deepseek-chat
任务类型: TECHNICAL


执行结果 - DeepSeek技术专家


二分查找（Binary Search）是一种高效的查找算法，适用于在已排序的数组中查找特定元素。它的基本思想是通过将查找区间逐步对半分割，从而快速缩小查找范围，直到找到目标元素或确定目标元素不存在。

### 时间复杂度
- **时间复杂度**：O(log n)，其中 n 是数组的长度。每次查找都将查找范围减半，因此时间复杂度是对数级别。
- **空间复杂度**：O(1)，因为二分查找是原地算法，不需要额外的空间。

### Python 实现

```python
def binary_search(arr, target):
    """
    在已排序的数组 arr 中查找目标元素 target。
    
    参数:
    arr (list): 已排序的数组
    target (int or float): 要查找的目标元素
    
    返回:
    int: 目标元素在数组中的索引，如果不存在则返回 -1
    """
    # 定义查找区间的左右边界
    left, right = 0, len(arr) - 1
    
    # 当左边界小于等于右边界时，继续查找
    while left <= right:
        # 计算中间位置的索引
        mid = (left + right) // 2
        
        # 如果中间位置的元素等于目标值，返回索引
        if arr[mid] == target:
            return mid
        
        # 如果中间位置的元素小于目标值，说明目标值在右半部分，更新左边界
        elif arr[mid] < target:
            left = mid + 1
        
        # 如果中间位置的元素大于目标值，说明目标值在左半部分，更新右边界
        else:
            right = mid - 1
    
    # 如果查找区间已经缩小到空，说明目标元素不存在，返回 -1
    return -1

# 测试代码
if __name__ == "__main__":
    arr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    target = 7
    result = binary_search(arr, target)
    
    if result != -1:
        print(f"元素 {target} 在数组中的索引为 {result}")
    else:
        print(f"元素 {target} 不在数组中")
```

### 代码说明

1. **参数说明**：
   - `arr`: 已排序的数组。
   - `target`: 要查找的目标元素。

2. **初始化左右边界**：
   - `left` 初始化为 `0`，表示数组的起始位置。
   - `right` 初始化为 `len(arr) - 1`，表示数组的结束位置。

3. **循环查找**：
   - 在 `while` 循环中，计算中间位置 `mid`。
   - 如果 `arr[mid]` 等于 `target`，则返回 `mid`，表示找到了目标元素。
   - 如果 `arr[mid]` 小于 `target`，说明目标元素在右半部分，更新左边界 `left` 为 `mid + 1`。
   - 如果 `arr[mid]` 大于 `target`，说明目标元素在左半部分，更新右边界 `right` 为 `mid - 1`。

4. **查找失败**：
   - 如果 `left > right`，说明查找区间已缩小到空，目标元素不存在于数组中，返回 `-1`。

### 示例输出

```
元素 7 在数组中的索引为 6
```

### 总结

二分查找算法通过逐步缩小查找范围，能够高效地在已排序的数组中查找目标元素。其时间复杂度为 O(log n)，是处理大规模数据集时常用的查找方法之一。

## 7、测试案例2：创意类任务

测试一个创意写作任务，观察Router如何将其路由到Claude模型

In [8]:
print("\n" + "="*70)
print("测试案例2：创意类任务")
print("="*70)

# 创意类请求
creative_request = """请为一家新开业的咖啡店写一段温馨的宣传文案，
强调手工烘焙、温暖氛围和社区连接，字数200字左右。"""

# 步骤1: Router分析任务
task_type_2, reasoning_2 = route_task(creative_request)

# 步骤2: 执行任务
result_2, model_2, name_2 = execute_task(creative_request, task_type_2)

# 显示结果
print(f"\n{'='*70}")
print(f"执行结果 - {name_2}")
print("="*70)
display(Markdown(result_2))


测试案例2：创意类任务

=== Router 正在分析任务 ===
用户请求: 请为一家新开业的咖啡店写一段温馨的宣传文案，
强调手工烘焙、温暖氛围和社区连接，字数200字左右。

Router模型 (gpt-5-mini) 的原始输出:
{
  "task_type": "creative",
  "confidence": "99",
  "reasoning": "用户明确要求“写一段温馨的宣传文案”，并指定语气（温馨）、重点（手工烘焙、温暖氛围、社区连接）和字数约200字，属于文案写作与营销内容创作，属于创意类任务而非技术类。"
}

✓ 路由决策: CREATIVE
✓ 置信度: 99%
✓ 理由: 用户明确要求“写一段温馨的宣传文案”，并指定语气（温馨）、重点（手工烘焙、温暖氛围、社区连接）和字数约200字，属于文案写作与营销内容创作，属于创意类任务而非技术类。

=== 使用专门模型处理任务 ===
专门模型: Claude创意专家
模型ID: anthropic/claude-sonnet-4-5
任务类型: CREATIVE


执行结果 - Claude创意专家


# 一杯咖啡，一份温暖相遇

**在这里，每一颗咖啡豆都有故事**

我们用双手精心烘焙每一粒咖啡豆，在清晨的第一缕阳光中，聆听它们在烘焙机里欢快起舞的声音。从生豆到熟豆，从研磨到萃取，每个步骤都倾注着我们对咖啡的热爱与执着。

**这不只是一家咖啡店，更是社区的温暖客厅**

木质的桌椅、柔和的灯光、空气中弥漫的咖啡香气，我们想营造的，是让您卸下疲惫的港湾。邻居们在这里分享生活点滴，陌生人因一杯咖啡成为朋友，孩子们的笑声与咖啡机的蒸汽声交织成最美的乐章。

**欢迎回家**

无论您是需要一个安静的角落工作，还是想与朋友倾心畅谈，亦或只是路过想歇歇脚，这里的门永远为您敞开。

*让我们用一杯手工咖啡，温暖您的每一天。*

## 8、批量测试多个任务

测试多个不同类型的任务，展示Router的智能路由能力

In [9]:
# 定义多个测试任务
test_tasks = [
    "帮我分析一下快速排序和归并排序的优缺点",
    "写一首关于春天的小诗",
    "如何优化Python代码的性能？",
    "为儿童玩具品牌设计一个创意广告语"
]

print("\n" + "="*70)
print("批量任务路由测试")
print("="*70)

routing_summary = []

for idx, task in enumerate(test_tasks, 1):
    print(f"\n{'='*70}")
    print(f"任务 {idx}: {task}")
    print("="*70)
    
    # 路由任务
    task_type, reasoning = route_task(task)
    specialist = SPECIALIST_MODELS[task_type]
    
    routing_summary.append({
        "task": task,
        "routed_to": task_type,
        "model": specialist["model"],
        "specialist_name": specialist["name"],
        "reasoning": reasoning
    })

print(f"\n{'='*70}")
print("路由汇总")
print("="*70)


批量任务路由测试

任务 1: 帮我分析一下快速排序和归并排序的优缺点

=== Router 正在分析任务 ===
用户请求: 帮我分析一下快速排序和归并排序的优缺点

Router模型 (gpt-5-mini) 的原始输出:
{
  "task_type": "technical",
  "confidence": "95",
  "reasoning": "用户请求要求分析快速排序和归并排序的优缺点，属于算法和计算机科学范畴，涉及时间复杂度、空间复杂度、稳定性、实现细节和工程权衡等技术性内容，因此判定为technical。意图明确且无歧义，故置信度较高。"
}

✓ 路由决策: TECHNICAL
✓ 置信度: 95%
✓ 理由: 用户请求要求分析快速排序和归并排序的优缺点，属于算法和计算机科学范畴，涉及时间复杂度、空间复杂度、稳定性、实现细节和工程权衡等技术性内容，因此判定为technical。意图明确且无歧义，故置信度较高。

任务 2: 写一首关于春天的小诗

=== Router 正在分析任务 ===
用户请求: 写一首关于春天的小诗

Router模型 (gpt-5-mini) 的原始输出:
{
  "task_type": "creative",
  "confidence": "98",
  "reasoning": "用户请求“写一首关于春天的小诗”属于文学写作与创意表达，涉及诗歌创作而非编程、算法或技术实现，因此应归类为 creative。"
}

✓ 路由决策: CREATIVE
✓ 置信度: 98%
✓ 理由: 用户请求“写一首关于春天的小诗”属于文学写作与创意表达，涉及诗歌创作而非编程、算法或技术实现，因此应归类为 creative。

任务 3: 如何优化Python代码的性能？

=== Router 正在分析任务 ===
用户请求: 如何优化Python代码的性能？

Router模型 (gpt-5-mini) 的原始输出:
{
  "task_type": "technical",
  "confidence": "95",
  "reasoning": "用户询问“如何优化Python代码的性能”，明确涉及编程语言和性能优化技术（如算法、数据结构、剖析、并发、内存管理等），属于技术类问题而非创意写作或文案创作。"
}

✓ 

## 9、可视化路由统计

In [10]:
# 统计路由分布
technical_count = sum(1 for item in routing_summary if item["routed_to"] == "technical")
creative_count = sum(1 for item in routing_summary if item["routed_to"] == "creative")

statistics = f"""
### 路由统计报告

**总任务数**: {len(routing_summary)}

**路由分布**:
- 技术类任务 (DeepSeek): {technical_count} 个 ({technical_count/len(routing_summary)*100:.1f}%)
- 创意类任务 (Claude): {creative_count} 个 ({creative_count/len(routing_summary)*100:.1f}%)

**详细路由记录**:
"""

for idx, item in enumerate(routing_summary, 1):
    route_icon = "💻" if item["routed_to"] == "technical" else "✍️"
    statistics += f"\n{idx}. {route_icon} **{item['task'][:40]}...**"
    statistics += f"\n   - 路由到: {item['specialist_name']}"
    statistics += f"\n   - 理由: {item['reasoning']}\n"

display(Markdown(statistics))


### 路由统计报告

**总任务数**: 4

**路由分布**:
- 技术类任务 (DeepSeek): 2 个 (50.0%)
- 创意类任务 (Claude): 2 个 (50.0%)

**详细路由记录**:

1. 💻 **帮我分析一下快速排序和归并排序的优缺点...**
   - 路由到: DeepSeek技术专家
   - 理由: 用户请求要求分析快速排序和归并排序的优缺点，属于算法和计算机科学范畴，涉及时间复杂度、空间复杂度、稳定性、实现细节和工程权衡等技术性内容，因此判定为technical。意图明确且无歧义，故置信度较高。

2. ✍️ **写一首关于春天的小诗...**
   - 路由到: Claude创意专家
   - 理由: 用户请求“写一首关于春天的小诗”属于文学写作与创意表达，涉及诗歌创作而非编程、算法或技术实现，因此应归类为 creative。

3. 💻 **如何优化Python代码的性能？...**
   - 路由到: DeepSeek技术专家
   - 理由: 用户询问“如何优化Python代码的性能”，明确涉及编程语言和性能优化技术（如算法、数据结构、剖析、并发、内存管理等），属于技术类问题而非创意写作或文案创作。

4. ✍️ **为儿童玩具品牌设计一个创意广告语...**
   - 路由到: Claude创意专家
   - 理由: 用户请求是“为儿童玩具品牌设计一个创意广告语”，这是典型的文案/营销/创意写作任务，不涉及编程、算法或技术分析，属于创意类任务。


## 10、Routing模式流程图与总结

In [11]:
flow_diagram = """
```
┌────────────────────────────────────────────────────────────────┐
│                      Routing 模式流程图                         │
└────────────────────────────────────────────────────────────────┘

                    ┌──────────────────┐
                    │   用户请求输入    │
                    └────────┬─────────┘
                             │
                             ▼
                    ┌──────────────────┐
                    │  Router (GPT)    │  ◄── 分析任务类型
                    │   gpt-5-mini     │      判断路由方向
                    └────────┬─────────┘
                             │
                ┌────────────┴────────────┐
                │                         │
                ▼                         ▼
       ┌─────────────────┐       ┌─────────────────┐
       │  技术类任务？    │       │  创意类任务？    │
       └────────┬────────┘       └────────┬────────┘
                │                         │
                ▼                         ▼
       ┌─────────────────┐       ┌─────────────────┐
       │   DeepSeek专家  │       │   Claude专家    │
       │  deepseek-chat  │       │ claude-sonnet   │
       └────────┬────────┘       └────────┬────────┘
                │                         │
                │  💻 编程/技术            │  ✍️  创意/写作
                │                         │
                └────────────┬────────────┘
                             │
                             ▼
                    ┌──────────────────┐
                    │   返回处理结果    │
                    └──────────────────┘
```

### Routing 模式的核心优势

1. **专业化分工**
   - 每个模型专注于其擅长的领域
   - 提高任务处理质量和效率

2. **智能决策**
   - Router自动分析任务特征
   - 无需人工判断和分配

3. **关注点分离**
   - 路由逻辑与任务执行解耦
   - 易于扩展和维护

4. **灵活扩展**
   - 可轻松添加新的专门模型
   - 支持更多任务类型

5. **成本优化**
   - 根据任务复杂度选择合适模型
   - 避免大材小用

### 适用场景

- **多领域服务平台**: 需要处理不同类型用户请求
- **企业AI助手**: 涵盖技术支持、内容创作等多种功能
- **智能客服**: 根据问题类型分配到专门处理队列
- **内容生产**: 技术文档vs营销文案的自动分流
"""

display(Markdown(flow_diagram))


```
┌────────────────────────────────────────────────────────────────┐
│                      Routing 模式流程图                         │
└────────────────────────────────────────────────────────────────┘

                    ┌──────────────────┐
                    │   用户请求输入    │
                    └────────┬─────────┘
                             │
                             ▼
                    ┌──────────────────┐
                    │  Router (GPT)    │  ◄── 分析任务类型
                    │   gpt-5-mini     │      判断路由方向
                    └────────┬─────────┘
                             │
                ┌────────────┴────────────┐
                │                         │
                ▼                         ▼
       ┌─────────────────┐       ┌─────────────────┐
       │  技术类任务？    │       │  创意类任务？    │
       └────────┬────────┘       └────────┬────────┘
                │                         │
                ▼                         ▼
       ┌─────────────────┐       ┌─────────────────┐
       │   DeepSeek专家  │       │   Claude专家    │
       │  deepseek-chat  │       │ claude-sonnet   │
       └────────┬────────┘       └────────┬────────┘
                │                         │
                │  💻 编程/技术            │  ✍️  创意/写作
                │                         │
                └────────────┬────────────┘
                             │
                             ▼
                    ┌──────────────────┐
                    │   返回处理结果    │
                    └──────────────────┘
```

### Routing 模式的核心优势

1. **专业化分工**
   - 每个模型专注于其擅长的领域
   - 提高任务处理质量和效率

2. **智能决策**
   - Router自动分析任务特征
   - 无需人工判断和分配

3. **关注点分离**
   - 路由逻辑与任务执行解耦
   - 易于扩展和维护

4. **灵活扩展**
   - 可轻松添加新的专门模型
   - 支持更多任务类型

5. **成本优化**
   - 根据任务复杂度选择合适模型
   - 避免大材小用

### 适用场景

- **多领域服务平台**: 需要处理不同类型用户请求
- **企业AI助手**: 涵盖技术支持、内容创作等多种功能
- **智能客服**: 根据问题类型分配到专门处理队列
- **内容生产**: 技术文档vs营销文案的自动分流


## 11、保存路由结果

In [ ]:
# 保存路由摘要
routing_results = {
    "specialist_models": SPECIALIST_MODELS,
    "routing_summary": routing_summary,
    "statistics": {
        "total_tasks": len(routing_summary),
        "technical_count": technical_count,
        "creative_count": creative_count
    },
    "test_cases": [
        {
            "case": 1,
            "request": technical_request,
            "routed_to": task_type_1,
            "model": model_1,
            "result_length": len(result_1)
        },
        {
            "case": 2,
            "request": creative_request,
            "routed_to": task_type_2,
            "model": model_2,
            "result_length": len(result_2)
        }
    ]
}

with open('routing_results.json', 'w', encoding='utf-8') as f:
    json.dump(routing_results, f, ensure_ascii=False, indent=2)

print("✓ 路由结果已保存到 routing_results.json")
print(f"\n总共处理了 {len(routing_summary)} 个任务")
print(f"技术类路由: {technical_count} 次")
print(f"创意类路由: {creative_count} 次")